## Artificial Neural Networks Applications Using TensorFlow 
<br> Prepared for the Aritfical Neural Network Seminar at Columbia University, Sep 2017
>Author: Tristan Eisenhart
<br>te2252@columbia.edu

The code in this session was inspired by Aymeric Damien's great <a href="https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network_raw.ipynb"> post and work</a>.

If you haven't installed TensorFlow on your machine yet, please follow those intructions: https://www.tensorflow.org/install/

In [7]:
# Introduction to Neural Networks and TensorFlow -- currently using version 1.2 of TensorFlow
import tensorflow as tf

### Application 2: digit recognition

Problem here: we want to have a neural network recognize which digit correspond to which image. Best algorithms do better than the human eye nowadays. Why is this application interesting ? Because it will introduce us to batch training, multi-layer perceptrons and training and test sets splits.

<img src="https://camo.githubusercontent.com/b06741b45df8ffe29c7de999ab2ec4ff6b2965ba/687474703a2f2f6e657572616c6e6574776f726b73616e64646565706c6561726e696e672e636f6d2f696d616765732f6d6e6973745f3130305f6469676974732e706e67">

The first thing we will do is import the MNIST dataset. The MNIST dataset consists of handwritten digits that were produced by American Census Employees. For more information on this often used dataset, check out this <a href="https://en.wikipedia.org/wiki/MNIST_database"> link</a>. Since the dataset is used in the tutorials from TensorFlow we will conveniently import it from the tensorflow.examples.tutorials module. Notice that we use one hot encoding. One hot encoding is a way to express categorical variables using a binary format (you can read about one hot encoding in this great <a href="https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f">post</a>. Cool, now let's extract our input_data and store it into an "mnist" variable.

In [8]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Before doing anything, let's have a look at our data and print the first batch. The first feature is a 28 * 28 vector with the color intensity for each pixel in the image (I am only displaying 200 pixels in the below cell); the second array is the class of the image with a one-hot encoding system. The position of the "one" value in the array indicates the value of the digit represented on the image (this is a zero-based index, therefore numbers osciliate between 0-9).

In [9]:
# Let's look at our data
print("The second (zero-based index) feature of our first training observation is the class (notice the one hot encoding):\n",mnist.train.next_batch(1)[1])
print("\nThe first feature of our training observation is the pixel intensity for each 28*28 pixels:\n",mnist.train.next_batch(1)[0][0][200:400])

The second (zero-based index) feature of our first training observation is the class (notice the one hot encoding):
 [[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]

The first feature of our training observation is the pixel intensity for each 28*28 pixels:
 [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.1137255   0.33725491  0.          0.

With that said, one way of approaching this problem could be to use a KNN (K-Nearest Neighboor) to classify images. We could do that (and some ML tutorials online use that technique), but since this is a Deep Learning tutorial, let's use the very famous multi-layer perceptron (MLP). The MLP is a type of feed-forward neural network that has more than one hidden layer (in addition to the output layer). As you will see, it works particularly well on our digit classification task. If you want to read more about the MLP, follow this <a href="https://en.wikipedia.org/wiki/Multilayer_perceptron">link</a>.

Let's very quickly introduce the concept of batch training: when you train a deep learning model, it is important to split your data (randomly) into batches (i.e. subgroups of your training set). The reason for that is that your algorithm is approximating a function using an optimization method that learns at a certain learning rate. Therefore, at each step of the optimization, the gradient will take leaps at a certain rate (the learning rate) to "find" the true function that you are trying to learn. Feeding the entire training set to your algorithm at once would not be efficient to update your parametersm and thus the importance of batch training. You can find more information in this <a html="http://rishy.github.io/ml/2017/01/05/how-to-train-your-dnn/"> great post</a>. 

Coming up with the right batch size is not an exact science and requires the trial and error approach... Here, we will take a batch size of 100. We'll use a learning rate of 0.001 and two hidden layers with each 256 neurons. Our input data has 784 features (28*28) and 10 classes (digits from 0-9, one-hot encoded). We will display classification accuracy every 100 iterations. Our weights (one for each layer) and layers (also one for each layer) are initiated using gaussian random variables. We will use the softmax cross entropy loss function and the Adam Optimization algorithm. Notice how we calculate our accuracy (checking the percentage of correctly classified observations).

Notice how useful our placeholders are when using batch training: we simply use the feed_dict method when running our computational graph with a new batch at each iteration. 

So let's inititate our first step, and start by defining our graph:

In [10]:
# Resetting to default graph -- especially usefull when running multiple sessions
tf.reset_default_graph()

# Parameters
learning_rate = 0.001 # how quickly the model will learn in optimization methode
num_steps = 1000 # max number of iterations
batch_size = 100 # the size of the batch fed in a training iteration to the model
display_step = 100 # displaying results of the optimization every 100 iterations

num_hidden_1 = 256 # number of neurons in 1st layer
num_hidden_2 = num_hidden_1 # number of neurons in 2nd layer
num_input_features = 784 # MNIST data input (img shape: 28*28) -- equivalent to the number of features in the input dataset
num_classes = 10 # MNIST total classes (0-9 digits) -- number of classes in the output data

# Input Parameters
x = tf.placeholder(tf.float32, [None, num_input_features])
y = tf.placeholder(tf.float32, [None, num_classes])

# Store layers weight & bias -- initiating weights and biases using a normal distribution
weights = {
    'hidden_1': tf.Variable(tf.random_normal([num_input_features, num_hidden_1])),
    'hidden_2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'output': tf.Variable(tf.random_normal([num_hidden_2, num_classes]))}
biases = {
    'biases_1': tf.Variable(tf.random_normal([num_hidden_1])),
    'biases_2': tf.Variable(tf.random_normal([num_hidden_2])),
    'output': tf.Variable(tf.random_normal([num_classes]))}


# Let's create our 3 hidden layers
layer_1 = tf.add(tf.matmul(x, weights['hidden_1']), biases['biases_1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_2']), biases['biases_2'])
output = tf.matmul(layer_2, weights['output']) + biases['output']

# Define loss and optimizer -- here loss function is the cross entropy to which we apply the softmax function
# Softmax function is a normalized exponential funciton that transforms logits into a range from 0 to 1 and with sum if logits equal to 1
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)) # reduce mean is simply the mean of all losses

# Here using the Adam algorithm for optimization
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Let's now write step 2 and evaluate the graph that we have built + optimize the parameters of our network.

In [11]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    print("Starting the optimization!")
    
    for step in range(1, num_steps+1):
        
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        
        if step % display_step == 0 or step == 1:
            
            # Computing the loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x, y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))

    print("Optimization finished! :)")
    
    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images,y: mnist.test.labels}))

Starting the optimization!
Step 1, Minibatch Loss= 3912.4958, Training Accuracy= 0.160
Step 100, Minibatch Loss= 363.9111, Training Accuracy= 0.700
Step 200, Minibatch Loss= 162.6705, Training Accuracy= 0.860
Step 300, Minibatch Loss= 163.3880, Training Accuracy= 0.840
Step 400, Minibatch Loss= 70.5037, Training Accuracy= 0.930
Step 500, Minibatch Loss= 136.2056, Training Accuracy= 0.850
Step 600, Minibatch Loss= 91.5238, Training Accuracy= 0.880
Step 700, Minibatch Loss= 84.9496, Training Accuracy= 0.890
Step 800, Minibatch Loss= 114.7708, Training Accuracy= 0.850
Step 900, Minibatch Loss= 98.3015, Training Accuracy= 0.850
Step 1000, Minibatch Loss= 152.1535, Training Accuracy= 0.820
Optimization finished! :)
Testing Accuracy: 0.876


And there you have it. Play around with the hyperparameters of this network and see if you cna increase the prediciton accuracy on the test set. This exercise is a good first implementation of an image classification task in TensorFlow. For more information, go check Aymeric's <a href="https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network_raw.ipynb"> post</a>. You can also check out the official TensorFlow <a href="https://www.tensorflow.org/get_started/mnist/pros"> tutorial</a>.